In [1]:
# Source : https://github.com/tdeboissiere/DeepLearningImplementations/tree/master/GAN 
# Project Title : Convert GAN implementation into IPython Notebook

# Current Progress
# 1. Convert main model,training files
# 2. Create HDF5 dataset to be used for training
# 3. Trained network on CelebA dataset for 400 epochs
# 4. 
# 5. Add descriptions in code (WIP)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
graphviz is already the newest version (2.38.0-16ubuntu2).
libgraphviz-dev is already the newest version (2.38.0-16ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [2]:
!pip install -q keras
import keras

!pip install graphviz pydot
!apt-get install -y graphviz libgraphviz-dev pydot
import graphviz
import pydot
import matplotlib
import matplotlib.pyplot as plt

!pip install plotnine
from plotnine import ggplot

matplotlib.style.use('ggplot')
%matplotlib inline

Using TensorFlow backend.


/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [4]:
import os
import sys
import time

import keras.backend as K

import tensorflow as tf

from keras.models import Model
from keras.layers.core import Flatten, Dense, Dropout, Activation, Lambda, Reshape
from keras.layers.convolutional import Conv2D, Deconv2D, ZeroPadding2D, UpSampling2D
from keras.layers import Input, Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization

from keras.utils import generic_utils
from keras.optimizers import Adam, SGD
from keras.utils import plot_model
from keras.utils.generic_utils import Progbar

from IPython.core.debugger import set_trace

#!git clone https://github.com/siddharthalodha/mlsid.git
    
#!cp -r mlsid/models .
#!cp -r mlsid/figures .
#!cp -r mlsid/utils .

sys.path.append("./utils") # Utils
import general_utils
import data_utils #Being used for importing data,generating batches (CelebA Data is currently stored in an hdf5 file)
%matplotlib inline

#!mkdir /figures

# Define Hyperparameters for GAN

In [5]:
#Hyperparameters

backend="tensorflow" #Specify backend to be used
dset="celebA" #Specify dataset to be used, currently supported : mnist/celebA
generator="upsampling" #Generator to be used : upsampling/deconv
model_name="CNN" #Name of model
batch_size=32 #Batch size to be used 
n_batch_per_epoch=200 #Number of batches per epoch
nb_epoch=400 #Number of epochs
epoch=10 #Epoch size => Used for progress bars 
nb_classes=2 #Number of classes
do_plot=True #Plotting during execution
bn_mode=2 #Batch normalization
img_dim=64 #Dimension of image
label_smoothing="store_true" #Label smoothing
label_flipping=0 #Label flipping
noise_scale=0.5 
use_mbd="store_true"
image_data_format = "channels_last"

In [ ]:
# Downloading CelebA hdf5 dataset if not already downloaded

In [6]:
#Code to download CelebA hdf5 dataset from google drive
#!pip install PyDrive

#from google.colab import auth
#from oauth2client.client import GoogleCredentials
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive

#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)

#fileId = drive.CreateFile({'id': '1cUBPxqU-9Y6f_OAfPwdRmPg7ruZ2sfdk'})
#print(fileId['title'])  # CelebA Dataset
#fileId.GetContentFile('CelebA_64_data.h5')  # Save Drive file as a local file

#fileId = drive.CreateFile({'id': '10B5fIjTwgk1IyXNGbWTzYEXMxnbFixTJ'})
#print(fileId['title'])  # CelebA Dataset
#fileId.GetContentFile('MNIST.h5')  # Save Drive file as a local file


In [7]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
      raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
K.tensorflow_backend.set_session(tf.Session(config=config))

Found GPU at: /device:GPU:0


In [ ]:
# Define generator function#1 : Upsampling

In [8]:
def generator_upsampling(noise_dim, img_dim, bn_mode, model_name="generator_upsampling", dset="celebA"):
    """
    Generator model of the DCGAN
    args : img_dim (tuple of int) num_chan, height, width
           pretr_weights_file (str) file holding pre trained weights
    returns : model (keras NN) the Neural Net model
    """

    s = img_dim[1]
    f = 512

    if dset == "mnist":
        start_dim = int(s / 4)
        nb_upconv = 2
    else:
        start_dim = int(s / 16)
        nb_upconv = 4

    if K.image_data_format() == "channels_first":
        bn_axis = 1
        reshape_shape = (f, start_dim, start_dim)
        output_channels = img_dim[0]
    else:
        reshape_shape = (start_dim, start_dim, f)
        bn_axis = -1
        output_channels = img_dim[-1]

    gen_input = Input(shape=noise_dim, name="generator_input")

    x = Dense(f * start_dim * start_dim, input_dim=noise_dim)(gen_input)
    x = Reshape(reshape_shape)(x)
    x = BatchNormalization(axis=bn_axis)(x)
    x = Activation("relu")(x)

    # Upscaling blocks
    for i in range(nb_upconv):
        x = UpSampling2D(size=(2, 2))(x)
        nb_filters = int(f / (2 ** (i + 1)))
        x = Conv2D(nb_filters, (3, 3), padding="same")(x)
        x = BatchNormalization(axis=1)(x)
        x = Activation("relu")(x)
        x = Conv2D(nb_filters, (3, 3), padding="same")(x)
        x = Activation("relu")(x)

    x = Conv2D(output_channels, (3, 3), name="gen_Conv2D_final", padding="same", activation='tanh')(x)

    generator_model = Model(inputs=[gen_input], outputs=[x], name=model_name)

    return generator_model

In [ ]:
# Define generator function#2 : Deconvolution

In [ ]:
def generator_deconv(noise_dim, img_dim, bn_mode, batch_size, model_name="generator_deconv", dset="mnist"):
    """
    Generator model of the DCGAN
    args : nb_classes (int) number of classes
           img_dim (tuple of int) num_chan, height, width
           pretr_weights_file (str) file holding pre trained weights
    returns : model (keras NN) the Neural Net model
    """

    assert K.backend() == "tensorflow", "Deconv not implemented with theano"

    s = img_dim[1]
    f = 512

    if dset == "mnist":
        start_dim = int(s / 4)
        nb_upconv = 2
    else:
        start_dim = int(s / 16)
        nb_upconv = 4

    reshape_shape = (start_dim, start_dim, f)
    bn_axis = -1
    output_channels = img_dim[-1]

    gen_input = Input(shape=noise_dim, name="generator_input")

    x = Dense(f * start_dim * start_dim, input_dim=noise_dim)(gen_input)
    x = Reshape(reshape_shape)(x)
    x = BatchNormalization(axis=bn_axis)(x)
    x = Activation("relu")(x)

    # Transposed conv blocks
    for i in range(nb_upconv - 1):
        nb_filters = int(f / (2 ** (i + 1)))
        s = start_dim * (2 ** (i + 1))
        o_shape = (batch_size, s, s, nb_filters)
        x = Deconv2D(nb_filters, (3, 3), output_shape=o_shape, strides=(2, 2), padding="same")(x)
        x = BatchNormalization(axis=-1)(x)
        x = Activation("relu")(x)

    # Last block
    s = start_dim * (2 ** (nb_upconv))
    o_shape = (batch_size, s, s, output_channels)
    x = Deconv2D(output_channels, (3, 3), output_shape=o_shape, strides=(2, 2), padding="same")(x)
    x = Activation("tanh")(x)

    generator_model = Model(inputs=[gen_input], outputs=[x], name=model_name)

    return generator_model

In [ ]:
# Define DCGAN model

In [ ]:
def DCGAN(generator, discriminator_model, noise_dim, img_dim):

    noise_input = Input(shape=noise_dim, name="noise_input")

    generated_image = generator(noise_input)
    DCGAN_output = discriminator_model(generated_image)

    DCGAN = Model(inputs=[noise_input],
                  outputs=[DCGAN_output],
                  name="DCGAN")
    return DCGAN

In [ ]:
# Define loader function

In [ ]:
def load(model_name, noise_dim, img_dim, bn_mode, batch_size, dset="mnist", use_mbd=False):

    if model_name == "generator_upsampling":
        model = generator_upsampling(noise_dim, img_dim, bn_mode, model_name=model_name, dset=dset)
        model.summary()
        plot_model(model, to_file='./figures/%s.png' % model_name, show_shapes=True, show_layer_names=True)
        return model
    if model_name == "generator_deconv":
        model = generator_deconv(noise_dim, img_dim, bn_mode, batch_size, model_name=model_name, dset=dset)
        model.summary()
        plot_model(model, to_file='./figures/%s.png' % model_name, show_shapes=True, show_layer_names=True)
        return model
    if model_name == "DCGAN_discriminator":
        model = DCGAN_discriminator(noise_dim, img_dim, bn_mode, model_name=model_name, dset=dset, use_mbd=use_mbd)
        model.summary()
        plot_model(model, to_file='./figures/%s.png' % model_name, show_shapes=True, show_layer_names=True)
        return model

In [ ]:
# Define discriminator function

In [ ]:
def DCGAN_discriminator(noise_dim, img_dim, bn_mode, model_name="DCGAN_discriminator", dset="celebA", use_mbd=False):
    """
    Discriminator model of the DCGAN
    args : img_dim (tuple of int) num_chan, height, width
           pretr_weights_file (str) file holding pre trained weights
    returns : model (keras NN) the Neural Net model
    """

    if K.image_data_format() == "channels_first":
        bn_axis = 1
    else:
        bn_axis = -1

    disc_input = Input(shape=img_dim, name="discriminator_input")

    if dset == "mnist":
        list_f = [128]

    else:
        list_f = [64, 128, 256]

    # First conv
    x = Conv2D(32, (3, 3), strides=(2, 2), name="disc_Conv2D_1", padding="same")(disc_input)
    x = BatchNormalization(axis=bn_axis)(x)
    x = LeakyReLU(0.2)(x)

    # Next convs
    for i, f in enumerate(list_f):
        name = "disc_Conv2D_%s" % (i + 2)
        x = Conv2D(f, (3, 3), strides=(2, 2), name=name, padding="same")(x)
        x = BatchNormalization(axis=bn_axis)(x)
        x = LeakyReLU(0.2)(x)

    x = Flatten()(x)

    def minb_disc(x):
        diffs = K.expand_dims(x, 3) - K.expand_dims(K.permute_dimensions(x, [1, 2, 0]), 0)
        abs_diffs = K.sum(K.abs(diffs), 2)
        x = K.sum(K.exp(-abs_diffs), 2)

        return x

    def lambda_output(input_shape):
        return input_shape[:2]

    num_kernels = 100
    dim_per_kernel = 5

    M = Dense(num_kernels * dim_per_kernel, use_bias=False, activation=None)
    MBD = Lambda(minb_disc, output_shape=lambda_output)

    if use_mbd:
        x_mbd = M(x)
        x_mbd = Reshape((num_kernels, dim_per_kernel))(x_mbd)
        x_mbd = MBD(x_mbd)
        x = Concatenate(axis=bn_axis)([x, x_mbd])

    x = Dense(2, activation='softmax', name="disc_dense_2")(x)

    discriminator_model = Model(inputs=[disc_input], outputs=[x], name=model_name)

    return discriminator_model

In [9]:
def train_GAN(batch_size,n_batch_per_epoch,nb_epoch,generator,model_name,image_data_format,img_dim,bn_mode,label_smoothing,label_flipping,noise_scale,dset,use_mbd,epoch_size):
    # Setup environment (logging directory etc)
    #general_utils.setup_logging(model_name) #change this if setup_logging moved to separate file

    # Load and rescale data
    if dset == "celebA":
        X_real_train = data_utils.load_celebA(img_dim, image_data_format) #change func call if file moved
    if dset == "mnist":
        X_real_train, _, _, _ = data_utils.load_mnist(image_data_format) #change func call if file moved
    img_dim = X_real_train.shape[-3:]
    noise_dim = (100,)
    
    try:

        # Create optimizers
        opt_dcgan = Adam(lr=2E-4, beta_1=0.5, beta_2=0.999, epsilon=1e-08)
        opt_discriminator = SGD(lr=1E-3, momentum=0.9, nesterov=True)

        # Load generator model
        generator_model = load("generator_%s" % generator,
                                      noise_dim,
                                      img_dim,
                                      bn_mode,
                                      batch_size,
                                      dset=dset,
                                      use_mbd=use_mbd)
        
        # Load discriminator model
        discriminator_model = load("DCGAN_discriminator",
                                          noise_dim,
                                          img_dim,
                                          bn_mode,
                                          batch_size,
                                          dset=dset,
                                          use_mbd=use_mbd)

        generator_model.compile(loss='mse', optimizer=opt_discriminator)
        discriminator_model.trainable = False
        DCGAN_model = DCGAN(generator_model,
                                   discriminator_model,
                                   noise_dim,
                                   img_dim)
        
        loss = ['binary_crossentropy']
        loss_weights = [1]
        DCGAN_model.compile(loss=loss, loss_weights=loss_weights, optimizer=opt_dcgan)

        discriminator_model.trainable = True
        discriminator_model.compile(loss='binary_crossentropy', optimizer=opt_discriminator)

        gen_loss = 100
        disc_loss = 100

        # Start training
        print("Start training")
        for e in range(nb_epoch):
            #Initialize progbar and batch counter
            #progbar = Progbar(epoch_size)
            batch_counter = 1
            start = time.time()
            for X_real_batch in data_utils.gen_batch(X_real_train, batch_size):

                # Create a batch to feed the discriminator model
                X_disc, y_disc = data_utils.get_disc_batch(X_real_batch,
                                                           generator_model,
                                                           batch_counter,
                                                           batch_size,
                                                           noise_dim,
                                                           noise_scale=noise_scale,
                                                           label_smoothing=label_smoothing,
                                                           label_flipping=label_flipping)

                # Update the discriminator
                disc_loss = discriminator_model.train_on_batch(X_disc, y_disc)

                # Create a batch to feed the generator model
                X_gen, y_gen = data_utils.get_gen_batch(batch_size, noise_dim, noise_scale=noise_scale)

                # Freeze the discriminator
                discriminator_model.trainable = False
                gen_loss = DCGAN_model.train_on_batch(X_gen, y_gen)
                # Unfreeze the discriminator
                discriminator_model.trainable = True

                batch_counter += 1
                #progbar.add(batch_size, values=[("D logloss", disc_loss),
                #                                ("G logloss", gen_loss)])

                 #Save images for visualization
                if batch_counter % 100 == 0:
                    data_utils.plot_generated_batch(X_real_batch, generator_model,
                                                    batch_size, noise_dim, image_data_format)
                    
                if batch_counter >= n_batch_per_epoch:
                    break

            print("")
            print('Epoch %s/%s, Time: %s, Discriminator loss:%s,Generator loss:%s' % (e + 1, nb_epoch, time.time() - start,disc_loss,gen_loss))

            #Save weights for generator,discriminator and DCGAN
            if e % 5 == 0:
                gen_weights_path = os.path.join('./models/%s/gen_weights_epoch%s.h5' % (model_name, e))
                generator_model.save_weights(gen_weights_path, overwrite=True)

                disc_weights_path = os.path.join('./models/%s/disc_weights_epoch%s.h5' % (model_name, e))
                discriminator_model.save_weights(disc_weights_path, overwrite=True)

                DCGAN_weights_path = os.path.join('./models/%s/DCGAN_weights_epoch%s.h5' % (model_name, e))
                DCGAN_model.save_weights(DCGAN_weights_path, overwrite=True)
    except Exception as e:
            print(e)

In [10]:
os.environ["KERAS_BACKEND"] = "tensorflow" #Set backend to tensorflow
image_data_format = "channels_last" #Setting image data format for tensorflow
K.set_image_data_format(image_data_format)    
# Launch training
train_GAN(batch_size,n_batch_per_epoch,nb_epoch,generator,model_name,image_data_format,img_dim,bn_mode,label_smoothing,label_flipping,noise_scale,dset,use_mbd,epoch)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generator_input (InputLayer) (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 8192)              827392    
_________________________________________________________________
reshape_1 (Reshape)          (None, 4, 4, 512)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 4, 4, 512)         2048      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 4, 512)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 8, 8, 512)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 256)         1179904   
__________

Start training

Epoch 1/400, Time: 33.41769456863403, Discriminator loss:1.2269685,Generator loss:1.3847432

Epoch 2/400, Time: 26.528018474578857, Discriminator loss:0.97709376,Generator loss:1.9092624

Epoch 3/400, Time: 26.492492198944092, Discriminator loss:1.2060373,Generator loss:1.3501065

Epoch 4/400, Time: 26.28626036643982, Discriminator loss:0.9497103,Generator loss:1.2151648

Epoch 5/400, Time: 26.5520977973938, Discriminator loss:0.91608214,Generator loss:1.2114583

Epoch 6/400, Time: 26.311431407928467, Discriminator loss:1.117691,Generator loss:1.5738089

Epoch 7/400, Time: 26.29346466064453, Discriminator loss:1.1781392,Generator loss:1.4651277

Epoch 8/400, Time: 26.701908588409424, Discriminator loss:0.9702659,Generator loss:2.085391

Epoch 9/400, Time: 26.22271490097046, Discriminator loss:1.0706524,Generator loss:1.3010399

Epoch 10/400, Time: 26.59523034095764, Discriminator loss:0.9522537,Generator loss:2.3209167

Epoch 11/400, Time: 26.389544010162354, Discrimina


Epoch 88/400, Time: 26.772599458694458, Discriminator loss:0.79167,Generator loss:2.7030032

Epoch 89/400, Time: 26.22167682647705, Discriminator loss:0.81501883,Generator loss:2.6270204

Epoch 90/400, Time: 26.435142993927002, Discriminator loss:0.83117306,Generator loss:3.1011834

Epoch 91/400, Time: 26.256758451461792, Discriminator loss:0.8554063,Generator loss:1.6948043

Epoch 92/400, Time: 26.724063634872437, Discriminator loss:0.75767374,Generator loss:2.7976346

Epoch 93/400, Time: 26.22831439971924, Discriminator loss:0.77659607,Generator loss:3.05973

Epoch 94/400, Time: 26.540515184402466, Discriminator loss:0.813272,Generator loss:2.975686

Epoch 95/400, Time: 26.377065420150757, Discriminator loss:0.8020232,Generator loss:2.4520545

Epoch 96/400, Time: 26.353790760040283, Discriminator loss:0.90616876,Generator loss:2.97692

Epoch 97/400, Time: 26.84109663963318, Discriminator loss:0.7447773,Generator loss:2.9339137

Epoch 98/400, Time: 26.321821212768555, Discriminator l


Epoch 174/400, Time: 27.47257423400879, Discriminator loss:0.7794107,Generator loss:2.5176098

Epoch 175/400, Time: 27.74468469619751, Discriminator loss:0.8559309,Generator loss:2.2982314

Epoch 176/400, Time: 27.001818656921387, Discriminator loss:0.8867333,Generator loss:2.7669954

Epoch 177/400, Time: 26.711982011795044, Discriminator loss:0.8429191,Generator loss:3.2414093

Epoch 178/400, Time: 26.579306840896606, Discriminator loss:0.7627411,Generator loss:2.616054

Epoch 179/400, Time: 27.638082027435303, Discriminator loss:0.91610897,Generator loss:2.071127

Epoch 180/400, Time: 28.224013328552246, Discriminator loss:0.83256614,Generator loss:3.8216934

Epoch 181/400, Time: 27.10747218132019, Discriminator loss:0.767642,Generator loss:3.0596056

Epoch 182/400, Time: 26.277152061462402, Discriminator loss:0.93134975,Generator loss:2.4659495

Epoch 183/400, Time: 27.326907634735107, Discriminator loss:0.8580449,Generator loss:1.6492671

Epoch 184/400, Time: 28.16417145729065, Di


Epoch 260/400, Time: 30.689592599868774, Discriminator loss:0.80581284,Generator loss:3.5368104

Epoch 261/400, Time: 29.147627592086792, Discriminator loss:0.7857642,Generator loss:2.9535532

Epoch 262/400, Time: 27.843136310577393, Discriminator loss:0.7407031,Generator loss:3.1959715

Epoch 263/400, Time: 26.362658500671387, Discriminator loss:0.8121822,Generator loss:2.8949065

Epoch 264/400, Time: 27.008450269699097, Discriminator loss:0.7836006,Generator loss:3.695775

Epoch 265/400, Time: 30.08004403114319, Discriminator loss:0.7771136,Generator loss:3.3301504

Epoch 266/400, Time: 29.427543878555298, Discriminator loss:0.74555045,Generator loss:3.2042289

Epoch 267/400, Time: 28.70937943458557, Discriminator loss:0.83942634,Generator loss:2.8229644

Epoch 268/400, Time: 30.050166130065918, Discriminator loss:0.8462744,Generator loss:2.4833374

Epoch 269/400, Time: 27.305888414382935, Discriminator loss:1.0224615,Generator loss:2.6308208

Epoch 270/400, Time: 26.586183547973633


Epoch 346/400, Time: 30.857418298721313, Discriminator loss:0.7641773,Generator loss:3.9036427

Epoch 347/400, Time: 29.073664903640747, Discriminator loss:0.7959327,Generator loss:3.301043

Epoch 348/400, Time: 30.63842272758484, Discriminator loss:0.7700819,Generator loss:3.106958

Epoch 349/400, Time: 29.31922674179077, Discriminator loss:0.7731363,Generator loss:3.1892605

Epoch 350/400, Time: 26.236090183258057, Discriminator loss:0.74015427,Generator loss:3.9940917

Epoch 351/400, Time: 27.108957767486572, Discriminator loss:0.79127276,Generator loss:3.3146486

Epoch 352/400, Time: 30.260005950927734, Discriminator loss:0.7679405,Generator loss:3.6783974

Epoch 353/400, Time: 29.442238807678223, Discriminator loss:0.76468176,Generator loss:3.972277

Epoch 354/400, Time: 30.34156322479248, Discriminator loss:0.73401463,Generator loss:3.4628246

Epoch 355/400, Time: 30.380213499069214, Discriminator loss:0.7633462,Generator loss:3.3763165

Epoch 356/400, Time: 29.582578659057617, 

In [11]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)